In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
#updated

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 
def calculate_angle1(a,b):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(b) # End
    if a[0]>b[0]:
        c[0]+=10.0
    else:
        c[0]-=10.0
    print(b)
    print(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 


In [9]:


# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('The Perfect Pull Up - Do it right! (online-video-cutter.com) (1).mp4')
# cap = cv2.VideoCapture('The Perfect Pull Up - Do it right! (online-video-cutter.com).mp4')
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 2048) #to 1280 x 720 
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1152)

# counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame=cv2.flip(frame,1)
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            rshoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            relbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            rwrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            rear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]
            
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y] 
            
            angle = calculate_angle(rshoulder, relbow, rwrist)
          
            v1 = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility
            v2 = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility
            v3 = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility
            v4 = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility

            if v1>=0.40 and v2>=0.40 and v3>=0.40:
                if angle < 40 and rear[1]<rwrist[1]:
                    stage = "up"
                if angle > 120 and stage =='up':
                    stage="down"
                    counter +=1

            else:
                if v1<0.40: 
                    cv2.putText(image, 'Right elbow not visible', (320,58), 2, 2, (0,0,0),2, cv2.LINE_AA)



            #LEFT HAND SIDE
            lshoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            lelbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            lwrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            lear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
            
            light_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            light_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y] 
            
            lngle = calculate_angle(lshoulder, lelbow, lwrist)
          
            l1 = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility
            l2 = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility
            l3 = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility
            l4 = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility

            if l1>=0.40 and l2>=0.40 and l3>=0.40:
                if lngle < 40 and lear[1]<lwrist[1]:
                    stage = "up"
                if lngle > 120 and stage =='up':
                    stage="down"
                    counter +=1

            else:
                if l1<0.40: 
                    cv2.putText(image, 'Left elbow not visible', (320,58), 2, 2, (0,0,0),2, cv2.LINE_AA)
                    
                    
        except:
            pass 
        
        # Render curl counter
        # Setup status box
        #print(np.multiply(wrist,[1280,720]))

        # Visibility things
        
        #print("elbow: ",landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].visibility)
        #print("shoulder",landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility)
        
        cv2.rectangle(image, (0,0), (300,75), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,65), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (160,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (120,65), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=1), 
                                mp_drawing.DrawingSpec(color=(255,255,0), thickness=1, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
